In [15]:
include("astrobee_se3_script.jl")
using Plots

N = 200
MAX_ITERATIONS_NB = 1e6

model = Astrobee()

Xp,Up = initialize_trajectory(model, N)
X, U  = copy(Xp) .+ 2., copy(Up) .+ 2.

SCPproblem = GuSTOProblem(model, N, Xp, Up)

x_dim, u_dim = model.x_dim, model.u_dim

Delta0, omega0, omegamax, 
    epsilon, rho0, rho1, 
    beta_succ, beta_fail, gamma_fail, 
    convergence_threshold = get_initial_gusto_parameters(model)

Delta = Delta0
omega = omega0

# X = zeros(x_dim, N)
# U = zeros(u_dim, N-1)
println("oh yeah")

oh yeah


In [16]:
# init_duals = get_initial_constraint_dual_variable(SCPproblem, model)

init_duals = [-0.0463827, -0.039217, -0.042717, -1.11196, -0.880066, -0.993014, -0.00602703, 0.000622719, 0.0031733, 5.16808e-5, -0.0045176, 0.000116599, 0.0123632]

13-element Array{Float64,1}:
 -0.0463827  
 -0.039217   
 -0.042717   
 -1.11196    
 -0.880066   
 -0.993014   
 -0.00602703 
  0.000622719
  0.0031733  
  5.16808e-5 
 -0.0045176  
  0.000116599
  0.0123632  

In [17]:
include("astrobee_se3_script.jl")
 
sol_newton = solve_shooting(SCPproblem, model, init_duals)
#x_shooting = solve_shooting_once(SCPproblem, model, sol_newton.zero)

sol_minpack.converged = false
sol_minpack.msg = "iteration is not making good progress, measured by improvement from last 10 iterations"
sol_minpack.return_code = 5


(Any[], Results of Nonlinear Solver Algorithm
 * Algorithm: Modified Powell (Expert)
 * Starting Point: [-0.0463827, -0.039217, -0.042717, -1.11196, -0.880066, -0.993014, -0.00602703, 0.000622719, 0.0031733, 5.16808e-5, -0.0045176, 0.000116599, 0.0123632]
 * Zero: [NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]
 * Inf-norm of residuals: NaN
 * Convergence: false
 * Message: iteration is not making good progress, measured by improvement from last 10 iterations
 * Total time: 2.356930 seconds
 * Function Calls: 0
 * Jacobian Calls (df/dx): 0)

In [ ]:
include("astrobee_se3_script.jl")

X_all, U_all, x_shooting_all = [], [], []
push!(X_all, copy(X))
push!(U_all, copy(U))

B_success = false
it = 0

while it<MAX_ITERATIONS_NB && 
        !(it!=0 && it!=1 && it!=2 && it!=3 && B_success && 
            convergence_metric(model,X,U,Xp,Up)<convergence_threshold) &&
        omega<omegamax
    println("-----------")
    println("Iteration $it")
    println("metric=$(convergence_metric(model,X,U,Xp,Up))")
    println("-----------")
    
    
#     omega, Delta = SCPproblem.omega, SCPproblem.Delta
    
    Xp = copy(X)
    Up = copy(U)
    
    model.f, model.A, model.B = compute_dynamics(model, Xp, Up)
    
    reset_problem(SCPproblem, model)
    set_parameters(SCPproblem, model, Xp, Up, omega, Delta)
    define_cost(SCPproblem, model)
    define_constraints(SCPproblem, model)
    
    JuMP.optimize!(SCPproblem.solver_model)    
    p0 = get_initial_constraint_dual_variable(SCPproblem, model)
    println("dual variables:", p0)
    # get solution
    X_sol = JuMP.value.(SCPproblem.X)
    U_sol = JuMP.value.(SCPproblem.U)
    
    # try shooting
    #x_shooting, sol_newton = solve_shooting(SCPproblem, model, p0)
#     x_shooting = solve_shooting_once(SCPproblem, model, p0)
    #push!(x_shooting_all, x_shooting)
    
    # -----------
    # GuSTO Logic
    if it > 3
        if is_in_trust_region(model, X_sol, U_sol, Xp, Up, Delta)
            rho = accuracy_ratio(SCPproblem, model, X_sol, U_sol, Xp, Up)

            if rho > rho1
                println("Reject solution.")
                Delta = beta_fail * Delta
                omega     = omega
                B_success = false

            else
                println("Accept solution.")
                X = copy(X_sol)
                U = copy(U_sol)
                B_success = true
                if rho < rho0
                    Delta = min(beta_succ*Delta, Delta0)
                else
                    Delta = Delta
                end
                if satisfies_state_inequality_constraints(SCPproblem, model, X_sol, U_sol, Xp, Up, Delta)
                    omega = omega0
                else
                    println("Solution does not satisfy state constraints, increasing omega.")
                    omega = gamma_fail * omega
                end
            end
        else
            println("Reject solution (Outside trust region)")
            Delta = Delta
            omega     = gamma_fail * omega
            B_success = false
        end
#         if convergence_metric(model,X,U,Xp,Up) <0.6
#             println("Convergence metric very small. Decreasing Delta.")
#             Delta = Delta0./(2^it)
#         end
            
    else # always accept first solution
        X = copy(X_sol)
        U = copy(U_sol)
    end
        
                    
    # -----------

    push!(X_all, copy(X))
    push!(U_all, copy(U))
                    
    
    it += 1
    
    diff_with_prev = norm(copy(X) - copy(Xp), Inf)
    println("x(k) - x(k-1) = $diff_with_prev")
    println("Parameters:")
    println("omega=$omega")
    println("delta=$Delta")
end
println("Converged")

-----------
Iteration 0
metric=94.24868789068172
-----------

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

dual variables:[-426.734, -629.956, -539.297, -254.813, -350.104, -307.677, -11.0495, -5.75689, -6.80383, -5.77579, -3.0672, -4.48646, -3.69728]
x(k) - x(k-1) = 2.6083892469357477
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 1
metric=142.03253079262203
-----------
dual variables:[-14.1826, -19.0609, -16.8474, -44.0818, -52.9001, -48.9508, -25.2136, 5.65673, 5.25792, 4.10249, -0.887249, 0.506088, 3.70798]
x(k) - x(k-1) = 2.134836565849444
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 2
metric=70.108853912001
-----------
dua

dual variables:[-0.690358, -0.658888, -0.674836, -6.23574, -5.73462, -5.9825, -0.0989891, -0.00473607, 0.0817746, 3.18041, -0.0938292, 0.00120336, 0.124585]
Accept solution.
x(k) - x(k-1) = 0.026161956634260908
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 28
metric=0.8084670174974499
-----------
dual variables:[-0.671299, -0.639816, -0.655736, -6.12085, -5.62057, -5.86786, -0.0949906, -0.00398266, 0.0788893, 2.89919, -0.090483, 0.00155175, 0.120059]
Accept solution.
x(k) - x(k-1) = 0.0244348248561278
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 29
metric=0.760278994209859
-----------
dual variables:[-0.653428, -0.621962, -0.637843, -6.01213, -5.51282, -5.75948, -0.0912739, -0.00326654, 0.0762354, 2.63739, -0.0874044, 0.00188236, 0.11586]
Accept solution.
x(k) - x(k-1) = 0.02286555865946971
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 30
metric=0.725963106226897
-----------
dual variables:[-0.636633, -0.60521, -0.621042, -5.90906, -5.41083, -5.65681, 

dual variables:[-0.406709, -0.37859, -0.392498, -4.38957, -3.92462, -4.15254, -0.0455456, 0.00561166, 0.0441717, 0.0194716, -0.0507267, 0.00579445, 0.063991]
Accept solution.
x(k) - x(k-1) = 0.00888413180576264
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 55
metric=0.3431007642335697
-----------
dual variables:[-0.401071, -0.373104, -0.386932, -4.34924, -3.88563, -4.11286, -0.044674, 0.0057414, 0.0435305, 0.00895463, -0.0500105, 0.00583936, 0.0629807]
Accept solution.
x(k) - x(k-1) = 0.008617897203007208
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 56
metric=0.336546918635676
-----------
dual variables:[-0.395611, -0.367792, -0.381542, -4.31, -3.84773, -4.07428, -0.0438392, 0.00586151, 0.0429128, 0.00480175, -0.0493214, 0.00587945, 0.0620112]
Accept solution.
x(k) - x(k-1) = 0.008363311423534858
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 57
metric=0.33020895369271613
-----------
dual variables:[-0.390318, -0.362648, -0.37632, -4.2718, -3.81087, -4.

dual variables:[-0.298779, -0.274288, -0.286327, -3.58319, -3.15064, -3.36211, -0.0306424, 0.00694483, 0.0323999, 0.000658159, -0.03769, 0.00592883, 0.0463324]
Accept solution.
x(k) - x(k-1) = 0.005354481281458989
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 82
metric=0.22405493011094782
-----------
dual variables:[-0.296019, -0.271644, -0.283624, -3.56147, -3.12996, -3.34091, -0.0303095, 0.00694355, 0.0321064, 0.000637087, -0.0373684, 0.00590931, 0.0459258]
Accept solution.
x(k) - x(k-1) = 0.005287549673019365
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 83
metric=0.22121580084232786
-----------
dual variables:[-0.29332, -0.269059, -0.280982, -3.54016, -3.10968, -3.32011, -0.0299861, 0.00694037, 0.0318192, 0.00061764, -0.0370539, 0.00588896, 0.0455301]
Accept solution.
x(k) - x(k-1) = 0.005223166743632213
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 84
metric=0.21848738826264155
-----------
dual variables:[-0.29068, -0.266531, -0.278397, -3.51924, -

dual variables:[-0.241089, -0.219304, -0.229982, -3.11242, -2.70458, -2.90371, -0.0241265, 0.00649361, 0.026191, 0.000375978, -0.0309064, 0.00523523, 0.0382114]
Accept solution.
x(k) - x(k-1) = 0.004036753339540455
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 109
metric=0.16917864105193048
-----------
dual variables:[-0.239469, -0.21777, -0.228405, -3.0986, -2.69155, -2.89029, -0.0239561, 0.00646775, 0.0260124, 0.000370918, -0.0307116, 0.00520674, 0.0379934]
Accept solution.
x(k) - x(k-1) = 0.004003048639863449
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 110
metric=0.1677773460344077
-----------
dual variables:[-0.237877, -0.216262, -0.226855, -3.08497, -2.67871, -2.87706, -0.0237893, 0.00644162, 0.0258364, 0.000366028, -0.0305198, 0.00517824, 0.0377795]
Accept solution.
x(k) - x(k-1) = 0.003969928858920779
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 111
metric=0.16641154370382596
-----------
dual variables:[-0.236312, -0.21478, -0.225332, -3.07154

...

dual variables:[-0.0514473, -0.0430231, -0.0471349, -1.18499, -0.935249, -1.05687, -0.00697078, 0.000857236, 0.0041184, 5.63393e-5, -0.00576808, 0.000314294, 0.0138687]
Accept solution.
x(k) - x(k-1) = 0.0004058016418445609
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 785
metric=0.0142699601545541
-----------
dual variables:[-0.0514108, -0.0429943, -0.0471023, -1.1845, -0.934868, -1.05644, -0.00696488, 0.000855675, 0.00411219, 5.63104e-5, -0.00576001, 0.000312987, 0.0138594]
Accept solution.
x(k) - x(k-1) = 0.00040525524923251677
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 786
metric=0.014261612133823416
-----------
dual variables:[-0.0513745, -0.0429656, -0.0470699, -1.18402, -0.934488, -1.05601, -0.006959, 0.000854119, 0.00410601, 5.62816e-5, -0.00575196, 0.000311684, 0.0138501]
Accept solution.
x(k) - x(k-1) = 0.0004047098453993092
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 787
metric=0.014253654752351783
-----------
dual variables:[-0.0513383,

dual variables:[-0.0505414, -0.0423119, -0.046329, -1.17272, -0.925727, -1.04601, -0.00682098, 0.000817968, 0.00396198, 5.56073e-5, -0.00556402, 0.000281379, 0.0136318]
Accept solution.
x(k) - x(k-1) = 0.00039191206410094104
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 811
metric=0.01411690040174487
-----------
dual variables:[-0.0505083, -0.0422861, -0.0462996, -1.17226, -0.925377, -1.04561, -0.00681537, 0.00081651, 0.00395616, 5.55799e-5, -0.0055564, 0.000280156, 0.0136229]
Accept solution.
x(k) - x(k-1) = 0.00039139079007483035
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 812
metric=0.014113269146471182
-----------
dual variables:[-0.0504752, -0.0422604, -0.0462704, -1.17181, -0.925027, -1.04521, -0.00680976, 0.000815056, 0.00395035, 5.55526e-5, -0.00554881, 0.000278936, 0.013614]
Accept solution.
x(k) - x(k-1) = 0.0003908704577127442
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 813
metric=0.014109654304171488
-----------
dual variables:[-0.050442

dual variables:[-0.0497163, -0.0416745, -0.0456003, -1.16129, -0.916958, -1.03595, -0.00667818, 0.000781248, 0.00381507, 5.49108e-5, -0.00537139, 0.00025054, 0.0134053]
Accept solution.
x(k) - x(k-1) = 0.0003786603328184701
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 837
metric=0.01407295304694637
-----------
dual variables:[-0.0496861, -0.0416513, -0.0455737, -1.16087, -0.916635, -1.03558, -0.00667282, 0.000779884, 0.0038096, 5.48847e-5, -0.0053642, 0.000249392, 0.0133968]
Accept solution.
x(k) - x(k-1) = 0.0003781629616436688
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 838
metric=0.014073104497519901
-----------
dual variables:[-0.0496559, -0.0416282, -0.0455472, -1.16045, -0.916312, -1.03521, -0.00666746, 0.000778522, 0.00380414, 5.48585e-5, -0.00535702, 0.000248248, 0.0133883]
Accept solution.
x(k) - x(k-1) = 0.00037766648586579077
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 839
metric=0.014073775197266165
-----------
dual variables:[-0.049625

dual variables:[-0.0489623, -0.0411014, -0.0449393, -1.15062, -0.908847, -1.02659, -0.00654185, 0.000746848, 0.00367688, 5.42449e-5, -0.00518927, 0.00022159, 0.0131884]
Accept solution.
x(k) - x(k-1) = 0.0003681047009183036
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 863
metric=0.014133381321493846
-----------
dual variables:[-0.0489346, -0.0410805, -0.0449151, -1.15022, -0.908547, -1.02625, -0.00653673, 0.000745569, 0.00367172, 5.42198e-5, -0.00518247, 0.000220512, 0.0131802]
Accept solution.
x(k) - x(k-1) = 0.00036800219502897136
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 864
metric=0.014137787324223001
-----------
dual variables:[-0.048907, -0.0410597, -0.044891, -1.14983, -0.908248, -1.0259, -0.00653161, 0.000744292, 0.00366659, 5.41948e-5, -0.00517567, 0.000219437, 0.0131721]
Accept solution.
x(k) - x(k-1) = 0.00036789974104678436
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 865
metric=0.014142169507059228
-----------
dual variables:[-0.04887

dual variables:[-0.0482708, -0.0405844, -0.0443375, -1.14061, -0.90131, -1.01786, -0.00641154, 0.000714566, 0.00354666, 5.36057e-5, -0.00501684, 0.000194368, 0.0129805]
Accept solution.
x(k) - x(k-1) = 0.00036605822932633814
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 889
metric=0.014289612261381954
-----------
dual variables:[-0.0482454, -0.0405656, -0.0443155, -1.14024, -0.90103, -1.01753, -0.00640664, 0.000713364, 0.00354181, 5.35816e-5, -0.00501039, 0.000193353, 0.0129727]
Accept solution.
x(k) - x(k-1) = 0.00036600594723212865
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 890
metric=0.014297197959569654
-----------
dual variables:[-0.04822, -0.0405468, -0.0442935, -1.13986, -0.900752, -1.01721, -0.00640175, 0.000712165, 0.00353696, 5.35575e-5, -0.00500396, 0.000192341, 0.0129648]
Accept solution.
x(k) - x(k-1) = 0.00036595338006506495
Parameters:
omega=1.0
delta=1000.0
-----------
Iteration 891
metric=0.014304971587723064
-----------
dual variables:[-0.0481

In [ ]:
gamma_fail

In [ ]:
xN = x_shooting[:,end]

for i = 1:x_dim
    Fi = model.x_final[i] - xN[i]
    @show Fi
end

In [ ]:

    for x_shooting in x_shooting_all
        if size(x_shooting,1) > 1
        @show minimum(x_shooting)
        end
end

In [ ]:
x_shooting_all

In [ ]:
include("astrobee_se3_script.jl")
#plot(X[1,:],X[2,:])

plot_solutions(SCPproblem, model, X_all[end-1:end], U_all[end-1:end]; x_shooting_all=x_shooting_all)

In [ ]:
X = X_all[end]
# Xs = x_shooting_all[2]
 fig = plot(X[1,:], X[2,:], X[3,:]; c=:blue, lims = (-0.7,0.7))
# plot!(fig, Xs[1,:], Xs[2,:], Xs[3,:]; c=:red)

xlims = (0,10)
zlims = (0,10)
fig

In [ ]:
idx = [1,2]
x = X_all[2][1,:]
y = X_all[2][2,:]
z = X_all[2][3,:]
# plot(x,  y)
plot(1:N, y)